In [1]:
import numpy as np
import torch as th


from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
ray.init(num_cpus=160, ignore_reinit_error=True)

2023-08-28 09:19:03,394	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.11.3
Ray version:,2.6.2
Dashboard:,http://127.0.0.1:8266


In [3]:
insmp = Backtester(20190103, 20210101)
otsmp = Backtester(20210101, 20211231)

num of factors:150
num of factors:150


In [4]:
factors = insmp.calc_factor()

calc_factor took 258 seconds


In [5]:
factors2 = otsmp.calc_factor()

calc_factor took 140 seconds


In [6]:
df_ic = insmp.calc_ic()
df_ic2 = otsmp.calc_ic()

calc_ic took 221 seconds
calc_ic took 106 seconds


In [7]:
insmp.df_ic.query("y1>.01 and y2>.015 and y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.042246,0.046690,0.060397
20230813143935_satd_lexpr08_lopt34_10_0322,0.019739,0.023186,0.032060
20230813143936_satd_lexpr08_lopt34_10_0714,0.020054,0.023814,0.034957
20230813145451_satd_lexpr08_lopt34_10_0856,0.045665,0.051074,0.071974
20230813145458_satd_lexpr08_lopt34_10_0262,0.039762,0.043808,0.061600
...,...,...,...
20230827133530_satd_lexpr10_lopt50_10_2726,0.036933,0.040749,0.058083
20230827133531_satd_lexpr10_lopt50_10_2189,0.034665,0.037770,0.046655
20230827133532_satd_lexpr10_lopt50_10_2349,0.037695,0.040817,0.047964
20230827133535_satd_lexpr10_lopt50_10_2084,0.023837,0.027611,0.037554


In [8]:
otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02")

,y1,y2,y5
20230811171800_satd_lexpr08_lopt34_10_0042,0.029942,0.032678,0.039297
20230813143935_satd_lexpr08_lopt34_10_0322,0.011797,0.013961,0.012473
20230813145451_satd_lexpr08_lopt34_10_0856,0.026510,0.029783,0.036930
20230813145458_satd_lexpr08_lopt34_10_0262,0.023295,0.025203,0.033990
20230813145501_satd_lexpr08_lopt34_10_0726,0.029467,0.029454,0.030696
...,...,...,...
20230827133530_satd_lexpr10_lopt50_10_2726,0.025786,0.028604,0.035774
20230827133531_satd_lexpr10_lopt50_10_2189,0.025736,0.028524,0.029190
20230827133532_satd_lexpr10_lopt50_10_2349,0.025632,0.026935,0.028289
20230827133535_satd_lexpr10_lopt50_10_2084,0.012625,0.014044,0.014675


In [9]:
selected_idx = otsmp.df_ic.query("y1>0.01 and y2>0.01 and y5>0.01 and @insmp.df_ic.y1>.01 and @insmp.df_ic.y2>.015 and @insmp.df_ic.y5>.02").index

In [10]:
feature_dict = dict(zip(otsmp.factor_names,otsmp.factors))

In [11]:
selected_sigs = [feature_dict[i] for i in selected_idx]

In [12]:
def calc_sig_corr(sigs):
    n = len(sigs)
    df_corr = np.ones((n,n))
    corrs = []
    for i in range(n):
        for j in range(i+1,n):
            ic = torch.nanmean(batch_pearsonr(sigs[i],sigs[j],))
            df_corr[i,j] = ic
            df_corr[j,i] = ic
    return df_corr
            

In [31]:
def _mp_batch_pearsonr(task):
    i = task[0]
    j = task[1]
    return th.nanmean(batch_pearsonr(selected_sigs[i],selected_sigs[j]))
def mp_calc_sig_corr(selected_sigs):
    n = len(selected_sigs)
    df_corr = np.ones((n,n))
    jobs = [(i, j) for i in range(n) for j in range(i + 1, n)]

    with ProcessPoolExecutor(max_workers=160) as executor:
        for corr, (i, j) in zip(executor.map(_mp_batch_pearsonr, jobs), jobs):
            df_corr[i, j] = corr
            df_corr[j, i] = corr  # Assuming correlation is symmetric
    return df_corr
df_corr = mp_calc_sig_corr(selected_sigs)

In [13]:
df_corr = calc_sig_corr(selected_sigs)

In [14]:
df_corr.shape

(109, 109)

In [15]:
df_corr = pd.DataFrame(df_corr)

In [16]:
lowcorr_idx = remove_high_corr_factors(df_corr.values)
print(len(lowcorr_idx))

78


In [17]:
final = [selected_idx[idx] for idx in lowcorr_idx]

In [18]:

# 设置显示最大列数
pd.set_option('display.max_columns', None)

# 设置显示最大行数
pd.set_option('display.max_rows', None)
finaldf =  otsmp.df_ic.loc[final]

In [19]:
print(finaldf)

                                                  y1        y2        y5
20230811171800_satd_lexpr08_lopt34_10_0042  0.029942  0.032678  0.039297
20230813143935_satd_lexpr08_lopt34_10_0322  0.011797  0.013961  0.012473
20230813145451_satd_lexpr08_lopt34_10_0856  0.026510  0.029783  0.036930
20230813145501_satd_lexpr08_lopt34_10_0726  0.029467  0.029454  0.030696
20230814092613_satd_lexpr08_lopt34_10_0788  0.030076  0.031085  0.033477
20230814092614_satd_lexpr08_lopt34_10_0177  0.010392  0.011457  0.016512
20230814092615_satd_lexpr08_lopt34_10_0018  0.022174  0.023574  0.024581
20230814092615_satd_lexpr08_lopt34_10_0143  0.022328  0.025780  0.027584
20230814102408_satd_lexpr15_lopt34_10_0319  0.026077  0.027266  0.035737
20230814175110_satd_lexpr15_lopt34_10_0269  0.025901  0.027344  0.033700
20230814175115_satd_lexpr15_lopt34_10_0852  0.035336  0.036327  0.041362
20230815095228_satd_lexpr10_lopt34_10_0762  0.021678  0.022744  0.028150
20230815095232_satd_lexpr10_lopt34_10_0094  0.03341

In [20]:
finaldf.describe()

,y1,y2,y5
count,78.000000,78.000000,78.000000
mean,0.022827,0.024610,0.027384
std,0.006474,0.006685,0.008370
min,0.010208,0.011457,0.011241
25%,0.017698,0.018768,0.020664
50%,0.023386,0.024880,0.028219
75%,0.026602,0.029123,0.033962
max,0.036442,0.038170,0.041362


In [21]:
fs  = selected_sigs
y1d = otsmp.calter.ret1d
y2d = otsmp.calter.ret2d
y5d = otsmp.calter.ret5d

In [22]:
q1d = [batch_topk(fs[i],y1d) for i in range(len(fs))]
q2d = [batch_topk(fs[i],y2d) for i in range(len(fs))]
q5d = [batch_topk(fs[i],y5d) for i in range(len(fs))]

In [23]:
q1d = [q1d[idx] for idx in lowcorr_idx]
q2d = [q2d[idx] for idx in lowcorr_idx]
q5d = [q5d[idx] for idx  in lowcorr_idx]

In [24]:
finaldf['q1d'] = q1d
finaldf['q2d'] = q2d
finaldf['q5d'] = q5d

In [25]:
finaldf

,y1,y2,y5,q1d,q2d,q5d
20230811171800_satd_lexpr08_lopt34_10_0042,0.029942,0.032678,0.039297,"(0.00025, 0.00023)","(0.00051, 0.00037)","(0.00126, 0.00175)"
20230813143935_satd_lexpr08_lopt34_10_0322,0.011797,0.013961,0.012473,"(0.00016, 0.00016)","(0.0004, 0.00035)","(0.00082, 0.00105)"
20230813145451_satd_lexpr08_lopt34_10_0856,0.026510,0.029783,0.036930,"(0.00028, -0.00033)","(0.00053, -0.00037)","(0.00159, 0.00044)"
20230813145501_satd_lexpr08_lopt34_10_0726,0.029467,0.029454,0.030696,"(0.00024, 3e-05)","(0.00043, -6e-05)","(0.00045, -0.00064)"
20230814092613_satd_lexpr08_lopt34_10_0788,0.030076,0.031085,0.033477,"(0.00017, -0.00019)","(0.00038, -0.00038)","(0.00082, -0.0006)"
20230814092614_satd_lexpr08_lopt34_10_0177,0.010392,0.011457,0.016512,"(-0.00035, -0.00059)","(-0.00046, -0.001)","(0.00017, -0.0003)"
20230814092615_satd_lexpr08_lopt34_10_0018,0.022174,0.023574,0.024581,"(8e-05, -6e-05)","(0.00018, -0.00019)","(0.00071, 0.00156)"
20230814092615_satd_lexpr08_lopt34_10_0143,0.022328,0.025780,0.027584,"(0.00025, -0.00101)","(0.00059, -0.00101)","(0.00133, -0.0004)"
20230814102408_satd_lexpr15_lopt34_10_0319,0.026077,0.027266,0.035737,"(-3e-05, -0.00114)","(-1e-05, -0.00143)","(0.00139, -0.00011)"
20230814175110_satd_lexpr15_lopt34_10_0269,0.025901,0.027344,0.033700,"(-1e-05, -0.00032)","(2e-05, -0.00052)","(0.00027, 0.00066)"


In [26]:
finaldf.to_csv('result.csv')

In [28]:
df_corr.iloc[lowcorr_idx,lowcorr_idx].to_csv('df_corr.csv')

In [29]:
df_corr.replace(1,0).describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108
count,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000,109.000000
mean,0.383316,0.346899,0.522930,0.524106,0.322677,0.444371,0.234649,0.479553,0.361226,0.389222,0.294151,0.511904,0.514209,0.548951,0.521809,0.376980,0.496588,0.383657,0.533932,0.491275,0.538100,0.562347,0.421754,0.334725,0.357063,0.514378,0.498444,0.503767,0.141782,0.526686,0.561746,0.446015,0.327403,0.395758,0.555242,0.519955,0.430534,0.567184,0.192402,0.405025,0.520671,0.373757,0.555120,0.562917,0.333527,0.358562,0.524937,0.425108,0.458070,0.548716,0.360204,0.472199,0.422632,0.290262,0.395509,0.388601,0.418858,0.449042,0.473248,0.361767,0.318930,0.349043,0.418062,0.457137,0.555484,0.429178,0.387478,0.517329,0.236462,0.386152,0.188329,0.433148,0.365251,0.186829,0.549840,0.209152,0.552219,0.386133,0.344834,0.153645,0.455830,0.533160,0.429704,0.562035,0.409953,0.296299,0.211358,0.224630,0.404886,0.424348,0.428087,0.483512,0.399601,0.312298,0.550695,0.340718,0.412629,0.511863,0.551945,0.346507,0.461254,0.446853,0.202359,0.518659,0.442624,0.347162,0.400790,0.296612,0.553013
std,0.184081,0.154128,0.166390,0.180505,0.105892,0.147442,0.183175,0.156692,0.154463,0.119595,0.141268,0.200109,0.170904,0.200738,0.156714,0.175330,0.171653,0.134383,0.179537,0.172059,0.183592,0.171788,0.139237,0.123761,0.138232,0.185708,0.147817,0.167330,0.099593,0.177087,0.176371,0.140905,0.181119,0.138735,0.187873,0.162567,0.149034,0.176587,0.070355,0.174388,0.211326,0.138885,0.174199,0.183596,0.133189,0.120873,0.175904,0.156996,0.133958,0.195340,0.159876,0.142873,0.142548,0.114272,0.132702,0.147903,0.142884,0.180485,0.150268,0.147670,0.167207,0.132548,0.153635,0.166491,0.169455,0.186212,0.132340,0.187328,0.096244,0.111921,0.096147,0.135271,0.156474,0.095072,0.202042,0.096823,0.191916,0.189014,0.102943,0.063391,0.159709,0.180503,0.134341,0.186547,0.152727,0.130422,0.072981,0.086791,0.164348,0.172009,0.152578,0.145048,0.120638,0.125644,0.177930,0.122944,0.132711,0.188629,0.191728,0.136735,0.164547,0.153308,0.069457,0.160001,0.143081,0.115285,0.146492,0.105662,0.199415
min,0.000000,-0.076892,0.000000,0.000000,0.000000,0.000000,-0.113299,0.000000,-0.148033,0.000000,-0.121817,0.000000,0.000000,0.000000,0.000000,-0.133113,-0.004620,-0.076755,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.148033,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.096922,0.000000,0.000000,-0.026876,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.045142,0.000000,0.000000,0.000